<a href="https://colab.research.google.com/github/TuanKiet04/FineTune_SML/blob/main/Ho_Xuan_Huong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.10.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        # Tạo ChatML format cho Qwen
        text = ""
        for message in messages:
            if message["role"] == "user":
                text += f"<|im_start|>user\n{message['content']}<|im_end|>\n"
            elif message["role"] == "assistant":
                text += f"<|im_start|>assistant\n{message['content']}<|im_end|>\n"

        # Thêm EOS token
        text += tokenizer.eos_token
        texts.append(text)

    return {"text": texts}
pass

from datasets import load_dataset

dataset = load_dataset("json", data_files="ho_xuan_huong_style_50.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

In [ ]:
print(dataset)
print(dataset[0])  # Xem mẫu đầu tiên

Dataset({
    features: ['messages', 'text'],
    num_rows: 73
})
{'messages': [{'role': 'user', 'content': 'AI là gì?'}, {'role': 'assistant', 'content': 'Trí tuệ nhân tạo gọi AI,\nMáy móc học hỏi biết vài ba điều.\nDữ liệu nhiều thì mới hiểu,\nNhư người học trò chăm siêu bài thầy.\nNgười ta dạy nó làm hay,\nChẳng qua cũng chỉ theo đầy lời người!'}], 'text': '<|im_start|>user\nAI là gì?<|im_end|>\n<|im_start|>assistant\nTrí tuệ nhân tạo gọi AI,\nMáy móc học hỏi biết vài ba điều.\nDữ liệu nhiều thì mới hiểu,\nNhư người học trò chăm siêu bài thầy.\nNgười ta dạy nó làm hay,\nChẳng qua cũng chỉ theo đầy lời người!<|im_end|>\n<|endoftext|>'}


In [ ]:
# Xem kết quả
print("\n=== Sample formatted text ===")
print(dataset[0]["text"])


=== Sample formatted text ===
<|im_start|>user
AI là gì?<|im_end|>
<|im_start|>assistant
Trí tuệ nhân tạo gọi AI,
Máy móc học hỏi biết vài ba điều.
Dữ liệu nhiều thì mới hiểu,
Như người học trò chăm siêu bài thầy.
Người ta dạy nó làm hay,
Chẳng qua cũng chỉ theo đầy lời người!<|im_end|>
<|endoftext|>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/73 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.266 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 73 | Num Epochs = 6 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Step,Training Loss
1,4.167100
2,4.334700
3,4.215900
4,4.247200
5,4.150200
6,4.096600
7,4.284100
8,3.884400
9,3.747500
10,4.553700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

207.2205 seconds used for training.
3.45 minutes used for training.
Peak reserved memory = 7.975 GB.
Peak reserved memory for training = 0.709 GB.
Peak reserved memory % of max memory = 54.101 %.
Peak reserved memory for training % of max memory = 4.81 %.


In [ ]:
FastLanguageModel.for_inference(model)

# Test với prompt theo style Hồ Xuân Hương
test_prompt = "<|im_start|>user\nDeep learning là gì?<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_special_tokens=True)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)

user
Deep learning là gì?
assistant
Deep learning như biển cả mênh mông,
Máy tính học hỏi, tự tìm hiểu lòng người.
Như con trẻ học biết qua chơi,
Nhìn nhiều nghe nhiều, ngày một thông minh lên.

Dữ liệu nhiều thì mới thành tài,
Máy móc luyện tập, ngày đêm không nghỉ ngơi.
Người ta dạy nó nhiều lắm,
Máy ấy biết nhiều, biết đủ mọi thứ!𬭶



In [ ]:
FastLanguageModel.for_inference(model)

# Test với prompt theo style Hồ Xuân Hương
test_prompt = "<|im_start|>user\nMua laptop thì cần quan tâm thông số nào?<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_special_tokens=True)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)

user
Mua laptop thì cần quan tâm thông số nào?
assistant
Thân em chẳng lớn mà cũng có bộ ba:
 CPU nhanh, RAM đủ, SSD tròn tròn.
 CPU là não, RAM là nhớ,
 SSD là nơi lưu trữ dữ liệu.

 Chọn cái nào? Chọn cả ba!
 Chẳng chọn một thứ, mà chọn cả ba!


In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')